In [1]:
##################
### Author: Adriano Fonzino. email: adriano.fonzino@uniba.it
##################
import pandas as pd
import numpy as np
import pysam, gzip, os
from datetime import datetime

In [2]:
# define util function
def extract_bonafide(wt_fp, ko_fp, wt_name, ko_name, wgs_fp):
    wt = pysam.TabixFile(wt_fp)
    ko = pysam.TabixFile(ko_fp)
    wgs = pysam.TabixFile(wgs_fp)
    bonafide = []
    start_time = datetime.now()
    with gzip.open(wt_fp) as wt_table:
        for c,s in enumerate(wt_table):
            site = s.decode("utf-8").rstrip().split("\t")
            if c % 50000000 == 0:
                print("Evaluated sites:", c)
            if site[0] == "Region":
                # store header
                header = site[:9]
            else:
                if site[0].startswith("chr"):
                    if site[0] != "chrM":
                        if site[2] == "A":
                            if site[4] != "-":
                                if int(site[4]) >= min_cov:
                                    if site[7] == "AG" and float(site[8]) >= 0.01:
                                        vector = eval(site[6])
                                        if vector[2] >= min_ag_subs:
                                            ko_query = [i for i in ko.fetch(site[0], int(site[1])-1, int(site[1]))]
                                            if len(ko_query) == 1:
                                                ko_query = ko_query[0].split("\t")
                                                if ko_query[4] != "-":
                                                    if int(ko_query[4]) >= min_cov:
                                                        wgs_query = [i for i in wgs.fetch(site[0], int(site[1])-1, int(site[1]))]
                                                        if len(wgs_query) == 1:
                                                            wgs_query = wgs_query[0].split("\t")
                                                            if wgs_query[9] != "-":
                                                                if int(wgs_query[9]) >= min_cov_wgs:
                                                                    whole_site = site[:9] + ko_query[:9] + wgs_query[0:4] + wgs_query[9:]
                                                                    bonafide.append(whole_site)
    print("Iteration on tabix outTables finished. Elapsed time: ", datetime.now()-start_time)
    columns=["wt_"+i for i in header]+["ko_"+i for i in header]+["g"+i for i in header]
    bonafide = pd.DataFrame(bonafide, columns=columns)
    
    # save to disk bonafide candidates
    output_file = os.path.join(output_folder, wt_name + "_vs_" + ko_name + ".bonafide_candidates.tsv")
    print("Save to disk bonafide candidates:", output_file)
    bonafide.to_csv(output_file, sep="\t", index=None)
    
    # load from disk to infer dtypes
    bonafide = pd.read_table(output_file)
    
    # drop unstranded
    bonafide = bonafide[(bonafide["wt_Strand"]!=2)&(bonafide["ko_Strand"]!=2)&(bonafide["gStrand"]!=2)].copy()
    
    # select concordand for strand for wt and ko
    # select only strand concordand sites
    bonafide = bonafide[bonafide["wt_Strand"] == bonafide["ko_Strand"]].copy()
    bonafide = bonafide[bonafide["ko_Strand"] == bonafide["gStrand"]].copy()
    bonafide.reset_index(inplace=True, drop=True)
    display(bonafide)
    pos = bonafide[(bonafide["wt_AllSubs"]=="AG")&(bonafide["ko_AllSubs"]=="-")&(bonafide["gAllSubs"]=="-")].copy()
    pos["Class"] = "Editing"
    pos["Class_binary"] = 1
    print("Pos:", pos.shape)
    neg = bonafide[(bonafide["wt_AllSubs"]=="AG")&(bonafide["ko_AllSubs"]=="AG")&(bonafide["gAllSubs"]=="AG")].copy()
    # select negs with 0.01 AG freq and min_ag_subs
    neg = neg.query("ko_Frequency >= 0.01")
    mask = []
    for cand in neg["ko_BaseCount[A,C,G,T]"]:
        vector = eval(cand)
        if vector[2] >= min_ag_subs:
            mask.append(True)
        else:
            mask.append(False)
    neg = neg[mask].copy()
    neg["Class"] = "Not-Editing"
    neg["Class_binary"] = 0
    print("Neg:", neg.shape)
    # merge pos and neg and add samples id
    bonafide_final = pd.concat([pos, neg])
    bonafide_final.reset_index(inplace=True, drop=True)
    bonafide_final["wt_sample"] = wt_name
    bonafide_final["ko_sample"] = ko_name
    
    display(bonafide_final)
    print(bonafide_final[["Class", "Class_binary"]].value_counts())
    
    # save to disk bonafide final
    output_file = os.path.join(output_folder, wt_name + "_vs_" + ko_name + ".bonafide_final.tsv")
    print("Save to disk bonafide final:", output_file)
    bonafide_final.to_csv(output_file, sep="\t", index=None)
    
    wt.close()
    ko.close()
    wgs.close()
    return bonafide_final

In [4]:
# define output folder
output_folder = "/lustre/bio_running/new_basecaller/REDINET_TEST_30_07_2024/REDInet/Package/Results/u87"

In [5]:
# define wgs tabix outTable file path
wgs_fp = "/lustre/home/pietrolucamazzacuva/U87/sra/SRR8670718/DnaRna_618043343/outTable_618043343.gz"

# define common filters
min_cov = 30
min_cov_wgs = 10
min_ag_subs = 2

In [6]:
# couple 1 (KO1 vs WT1) unique for u87 cell-line
wt_fp = "/lustre/home/pietrolucamazzacuva/U87/sra/SRR388226_SRR388227/DnaRna_853538513/outTable_853538513.gz"
ko_fp = "/lustre/home/pietrolucamazzacuva/U87/sra/SRR388228_SRR388229/DnaRna_921089530/outTable_921089530.gz"
wt_name = "SRR388226_SRR388227.WT" + f".{os.path.basename(wt_fp)}"
ko_name = "SRR388228_SRR388229.KO" + f".{os.path.basename(ko_fp)}"
print(wt_name)
print(ko_name)

couple1 = extract_bonafide(wt_fp, ko_fp, wt_name, ko_name, wgs_fp)
couple1

SRR388226_SRR388227.WT.outTable_853538513.gz
SRR388228_SRR388229.KO.outTable_921089530.gz
Evaluated sites: 0
Evaluated sites: 50000000
Evaluated sites: 100000000
Evaluated sites: 150000000
Evaluated sites: 200000000
Iteration on tabix outTables finished. Elapsed time:  0:05:51.761224
Save to disk bonafide candidates: /lustre/bio_running/new_basecaller/REDINET_TEST_30_07_2024/REDInet/Package/Results/u87/SRR388226_SRR388227.WT.outTable_853538513.gz_vs_SRR388228_SRR388229.KO.outTable_921089530.gz.bonafide_candidates.tsv


,wt_Region,wt_Position,wt_Reference,wt_Strand,wt_Coverage-q30,wt_MeanQ,"wt_BaseCount[A,C,G,T]",wt_AllSubs,wt_Frequency,ko_Region,...,ko_Frequency,gRegion,gPosition,gReference,gStrand,gCoverage-q30,gMeanQ,"gBaseCount[A,C,G,T]",gAllSubs,gFrequency
0,chrY,21152719,A,0,74,38.07,"[0, 0, 74, 0]",AG,1.00,chrY,...,1.0,chrY,21152719,A,0,41,41.00,"[0, 0, 41, 0]",AG,1.00
1,chrY,21153609,A,0,55,40.13,"[0, 0, 55, 0]",AG,1.00,chrY,...,1.0,chrY,21153609,A,0,34,37.68,"[0, 0, 34, 0]",AG,1.00
2,chrX,1460554,A,1,59,39.25,"[0, 0, 59, 0]",AG,1.00,chrX,...,1.0,chrX,1460554,A,1,22,36.68,"[0, 0, 22, 0]",AG,1.00
3,chrX,1508583,A,0,47,37.91,"[0, 0, 47, 0]",AG,1.00,chrX,...,1.0,chrX,1508583,A,0,24,38.67,"[1, 0, 23, 0]",AG,0.96
4,chrX,2139200,A,0,30,40.73,"[0, 0, 30, 0]",AG,1.00,chrX,...,1.0,chrX,2139200,A,0,22,35.41,"[0, 0, 22, 0]",AG,1.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10323,chr8,145650168,A,0,231,37.45,"[229, 0, 2, 0]",AG,0.01,chr8,...,0.0,chr8,145650168,A,0,57,35.44,"[57, 0, 0, 0]",-,0.00
10324,chr8,145651416,A,0,259,36.32,"[257, 0, 2, 0]",AG,0.01,chr8,...,0.0,chr8,145651416,A,0,51,32.61,"[51, 0, 0, 0]",-,0.00
10325,chr8,145651418,A,0,260,36.41,"[256, 0, 4, 0]",AG,0.02,chr8,...,0.0,chr8,145651418,A,0,51,33.35,"[51, 0, 0, 0]",-,0.00
10326,chr8,145693720,A,1,118,37.03,"[0, 0, 118, 0]",AG,1.00,chr8,...,1.0,chr8,145693720,A,1,46,32.70,"[0, 0, 46, 0]",AG,1.00


Pos: (5866, 29)
Neg: (1441, 29)


,wt_Region,wt_Position,wt_Reference,wt_Strand,wt_Coverage-q30,wt_MeanQ,"wt_BaseCount[A,C,G,T]",wt_AllSubs,wt_Frequency,ko_Region,...,gStrand,gCoverage-q30,gMeanQ,"gBaseCount[A,C,G,T]",gAllSubs,gFrequency,Class,Class_binary,wt_sample,ko_sample
0,chrX,16754311,A,1,146,37.49,"[144, 0, 2, 0]",AG,0.01,chrX,...,1,18,39.17,"[18, 0, 0, 0]",-,0.00,Editing,1,SRR388226_SRR388227.WT.outTable_853538513.gz,SRR388228_SRR388229.KO.outTable_921089530.gz
1,chrX,19368150,A,1,144,37.78,"[142, 0, 2, 0]",AG,0.01,chrX,...,1,17,35.59,"[17, 0, 0, 0]",-,0.00,Editing,1,SRR388226_SRR388227.WT.outTable_853538513.gz,SRR388228_SRR388229.KO.outTable_921089530.gz
2,chrX,19555884,A,0,187,37.07,"[185, 0, 2, 0]",AG,0.01,chrX,...,0,15,37.47,"[15, 0, 0, 0]",-,0.00,Editing,1,SRR388226_SRR388227.WT.outTable_853538513.gz,SRR388228_SRR388229.KO.outTable_921089530.gz
3,chrX,19610250,A,0,294,37.65,"[292, 0, 2, 0]",AG,0.01,chrX,...,0,23,35.48,"[23, 0, 0, 0]",-,0.00,Editing,1,SRR388226_SRR388227.WT.outTable_853538513.gz,SRR388228_SRR388229.KO.outTable_921089530.gz
4,chrX,23740019,A,0,101,37.06,"[99, 0, 2, 0]",AG,0.02,chrX,...,0,20,36.75,"[20, 0, 0, 0]",-,0.00,Editing,1,SRR388226_SRR388227.WT.outTable_853538513.gz,SRR388228_SRR388229.KO.outTable_921089530.gz
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7302,chr8,134470631,A,0,33,43.21,"[17, 0, 16, 0]",AG,0.48,chr8,...,0,45,35.93,"[27, 0, 18, 0]",AG,0.40,Not-Editing,0,SRR388226_SRR388227.WT.outTable_853538513.gz,SRR388228_SRR388229.KO.outTable_921089530.gz
7303,chr8,144991176,A,0,325,38.59,"[141, 0, 184, 0]",AG,0.57,chr8,...,0,41,31.78,"[16, 0, 25, 0]",AG,0.61,Not-Editing,0,SRR388226_SRR388227.WT.outTable_853538513.gz,SRR388228_SRR388229.KO.outTable_921089530.gz
7304,chr8,144992103,A,0,352,38.34,"[149, 0, 203, 0]",AG,0.58,chr8,...,0,33,35.52,"[18, 0, 15, 0]",AG,0.45,Not-Editing,0,SRR388226_SRR388227.WT.outTable_853538513.gz,SRR388228_SRR388229.KO.outTable_921089530.gz
7305,chr8,145150832,A,1,215,39.82,"[0, 0, 215, 0]",AG,1.00,chr8,...,1,30,34.30,"[0, 0, 30, 0]",AG,1.00,Not-Editing,0,SRR388226_SRR388227.WT.outTable_853538513.gz,SRR388228_SRR388229.KO.outTable_921089530.gz


Class        Class_binary
Editing      1               5866
Not-Editing  0               1441
dtype: int64
Save to disk bonafide final: /lustre/bio_running/new_basecaller/REDINET_TEST_30_07_2024/REDInet/Package/Results/u87/SRR388226_SRR388227.WT.outTable_853538513.gz_vs_SRR388228_SRR388229.KO.outTable_921089530.gz.bonafide_final.tsv


,wt_Region,wt_Position,wt_Reference,wt_Strand,wt_Coverage-q30,wt_MeanQ,"wt_BaseCount[A,C,G,T]",wt_AllSubs,wt_Frequency,ko_Region,...,gStrand,gCoverage-q30,gMeanQ,"gBaseCount[A,C,G,T]",gAllSubs,gFrequency,Class,Class_binary,wt_sample,ko_sample
0,chrX,16754311,A,1,146,37.49,"[144, 0, 2, 0]",AG,0.01,chrX,...,1,18,39.17,"[18, 0, 0, 0]",-,0.00,Editing,1,SRR388226_SRR388227.WT.outTable_853538513.gz,SRR388228_SRR388229.KO.outTable_921089530.gz
1,chrX,19368150,A,1,144,37.78,"[142, 0, 2, 0]",AG,0.01,chrX,...,1,17,35.59,"[17, 0, 0, 0]",-,0.00,Editing,1,SRR388226_SRR388227.WT.outTable_853538513.gz,SRR388228_SRR388229.KO.outTable_921089530.gz
2,chrX,19555884,A,0,187,37.07,"[185, 0, 2, 0]",AG,0.01,chrX,...,0,15,37.47,"[15, 0, 0, 0]",-,0.00,Editing,1,SRR388226_SRR388227.WT.outTable_853538513.gz,SRR388228_SRR388229.KO.outTable_921089530.gz
3,chrX,19610250,A,0,294,37.65,"[292, 0, 2, 0]",AG,0.01,chrX,...,0,23,35.48,"[23, 0, 0, 0]",-,0.00,Editing,1,SRR388226_SRR388227.WT.outTable_853538513.gz,SRR388228_SRR388229.KO.outTable_921089530.gz
4,chrX,23740019,A,0,101,37.06,"[99, 0, 2, 0]",AG,0.02,chrX,...,0,20,36.75,"[20, 0, 0, 0]",-,0.00,Editing,1,SRR388226_SRR388227.WT.outTable_853538513.gz,SRR388228_SRR388229.KO.outTable_921089530.gz
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7302,chr8,134470631,A,0,33,43.21,"[17, 0, 16, 0]",AG,0.48,chr8,...,0,45,35.93,"[27, 0, 18, 0]",AG,0.40,Not-Editing,0,SRR388226_SRR388227.WT.outTable_853538513.gz,SRR388228_SRR388229.KO.outTable_921089530.gz
7303,chr8,144991176,A,0,325,38.59,"[141, 0, 184, 0]",AG,0.57,chr8,...,0,41,31.78,"[16, 0, 25, 0]",AG,0.61,Not-Editing,0,SRR388226_SRR388227.WT.outTable_853538513.gz,SRR388228_SRR388229.KO.outTable_921089530.gz
7304,chr8,144992103,A,0,352,38.34,"[149, 0, 203, 0]",AG,0.58,chr8,...,0,33,35.52,"[18, 0, 15, 0]",AG,0.45,Not-Editing,0,SRR388226_SRR388227.WT.outTable_853538513.gz,SRR388228_SRR388229.KO.outTable_921089530.gz
7305,chr8,145150832,A,1,215,39.82,"[0, 0, 215, 0]",AG,1.00,chr8,...,1,30,34.30,"[0, 0, 30, 0]",AG,1.00,Not-Editing,0,SRR388226_SRR388227.WT.outTable_853538513.gz,SRR388228_SRR388229.KO.outTable_921089530.gz


In [7]:
# concat into a uniq dataset
bonafide_final_full = couple1
bonafide_final_full

,wt_Region,wt_Position,wt_Reference,wt_Strand,wt_Coverage-q30,wt_MeanQ,"wt_BaseCount[A,C,G,T]",wt_AllSubs,wt_Frequency,ko_Region,...,gStrand,gCoverage-q30,gMeanQ,"gBaseCount[A,C,G,T]",gAllSubs,gFrequency,Class,Class_binary,wt_sample,ko_sample
0,chrX,16754311,A,1,146,37.49,"[144, 0, 2, 0]",AG,0.01,chrX,...,1,18,39.17,"[18, 0, 0, 0]",-,0.00,Editing,1,SRR388226_SRR388227.WT.outTable_853538513.gz,SRR388228_SRR388229.KO.outTable_921089530.gz
1,chrX,19368150,A,1,144,37.78,"[142, 0, 2, 0]",AG,0.01,chrX,...,1,17,35.59,"[17, 0, 0, 0]",-,0.00,Editing,1,SRR388226_SRR388227.WT.outTable_853538513.gz,SRR388228_SRR388229.KO.outTable_921089530.gz
2,chrX,19555884,A,0,187,37.07,"[185, 0, 2, 0]",AG,0.01,chrX,...,0,15,37.47,"[15, 0, 0, 0]",-,0.00,Editing,1,SRR388226_SRR388227.WT.outTable_853538513.gz,SRR388228_SRR388229.KO.outTable_921089530.gz
3,chrX,19610250,A,0,294,37.65,"[292, 0, 2, 0]",AG,0.01,chrX,...,0,23,35.48,"[23, 0, 0, 0]",-,0.00,Editing,1,SRR388226_SRR388227.WT.outTable_853538513.gz,SRR388228_SRR388229.KO.outTable_921089530.gz
4,chrX,23740019,A,0,101,37.06,"[99, 0, 2, 0]",AG,0.02,chrX,...,0,20,36.75,"[20, 0, 0, 0]",-,0.00,Editing,1,SRR388226_SRR388227.WT.outTable_853538513.gz,SRR388228_SRR388229.KO.outTable_921089530.gz
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7302,chr8,134470631,A,0,33,43.21,"[17, 0, 16, 0]",AG,0.48,chr8,...,0,45,35.93,"[27, 0, 18, 0]",AG,0.40,Not-Editing,0,SRR388226_SRR388227.WT.outTable_853538513.gz,SRR388228_SRR388229.KO.outTable_921089530.gz
7303,chr8,144991176,A,0,325,38.59,"[141, 0, 184, 0]",AG,0.57,chr8,...,0,41,31.78,"[16, 0, 25, 0]",AG,0.61,Not-Editing,0,SRR388226_SRR388227.WT.outTable_853538513.gz,SRR388228_SRR388229.KO.outTable_921089530.gz
7304,chr8,144992103,A,0,352,38.34,"[149, 0, 203, 0]",AG,0.58,chr8,...,0,33,35.52,"[18, 0, 15, 0]",AG,0.45,Not-Editing,0,SRR388226_SRR388227.WT.outTable_853538513.gz,SRR388228_SRR388229.KO.outTable_921089530.gz
7305,chr8,145150832,A,1,215,39.82,"[0, 0, 215, 0]",AG,1.00,chr8,...,1,30,34.30,"[0, 0, 30, 0]",AG,1.00,Not-Editing,0,SRR388226_SRR388227.WT.outTable_853538513.gz,SRR388228_SRR388229.KO.outTable_921089530.gz


In [8]:
# save to disk bonafide final
output_file = os.path.join(output_folder, "bonafide_final_MERGED.tsv")
print("Save to disk bonafide final MERGED (all couples):", output_file)
bonafide_final_full.to_csv(output_file, sep="\t", index=None)

Save to disk bonafide final MERGED (all couples): /lustre/bio_running/new_basecaller/REDINET_TEST_30_07_2024/REDInet/Package/Results/u87/bonafide_final_MERGED.tsv


In [9]:
bonafide_final_full.Class.value_counts()

Editing        5866
Not-Editing    1441
Name: Class, dtype: int64

In [10]:
bonafide_final_full.groupby("Class")[["wt_Frequency", "ko_Frequency", "gFrequency"]].describe().T

Class                   Editing  Not-Editing
wt_Frequency count  5866.000000  1441.000000
             mean      0.037308     0.733685
             std       0.058338     0.284309
             min       0.010000     0.010000
             25%       0.010000     0.480000
             50%       0.020000     0.980000
             75%       0.040000     1.000000
             max       0.770000     1.000000
ko_Frequency count  5866.000000  1441.000000
             mean      0.000000     0.734254
             std       0.000000     0.284221
             min       0.000000     0.010000
             25%       0.000000     0.480000
             50%       0.000000     0.970000
             75%       0.000000     1.000000
             max       0.000000     1.000000
gFrequency   count  5866.000000  1441.000000
             mean      0.000000     0.742408
             std       0.000000     0.274093
             min       0.000000     0.020000
             25%       0.000000     0.500000
             50%       0.000000     0.970000
             75%       0.000000     1.000000
             max       0.000000     1.000000